In [ ]:
# IMPORT LIBRARIES

import numpy as np
import pandas as pd

In [1]:
# CLASS AND METHODS

class KinTimeInvariant:
    def __init__(self, p, f, birth_female=1/2.04, pi=None, output_kin=None):
        """
        Initialize the KinTimeInvariant class with the given parameters.

        Parameters:
        - p: Vector of survival probabilities.
        - f: Vector of fertility rates.
        - birth_female: Fraction of births that are female.
        - pi: Stationary distribution vector (optional).
        - output_kin: List of kin types to be included in the output (optional).
        """
        self.p = p
        self.f = f
        self.birth_female = birth_female
        self.pi = pi
        self.output_kin = output_kin

        self.ages = len(p)
        self.age = np.arange(self.ages)

        # Initialize matrices
        self.Ut, self.Mt, self.zeros = self._initialize_matrices()
        self.ft, self.e = self._initialize_fertility_matrix()

        # Calculate stationary distribution if not provided
        if self.pi is None:
            self.pi = self._calculate_stationary_distribution()

        # Initialize kin matrices
        self.kin_dict = self._initialize_kin_matrices()

        # Propagate kin over time
        self._propagate_kin()

        # Update kin dictionary to align dead kin and remove unwanted kin types
        self._update_kin_dict()

    def _initialize_matrices(self):
        """
        Initialize the transition matrices Ut and Mt.

        Returns:
        - Ut: Transition matrix for survival.
        - Mt: Transition matrix for mortality.
        - zeros: Zero matrix for block matrix operations.
        """
        Ut = np.zeros((self.ages, self.ages))
        Mt = np.zeros((self.ages, self.ages))
        zeros = np.zeros((self.ages, self.ages))

        for i in range(self.ages - 1):
            Ut[i + 1, i] = self.p[i]

        Ut[self.ages - 1, self.ages - 1] = self.p[self.ages - 1]
        np.fill_diagonal(Mt, 1 - np.array(self.p))
        Ut = np.block([
            [Ut, zeros],
            [Mt, zeros]
        ])

        return Ut, Mt, zeros

    def _initialize_fertility_matrix(self):
        """
        Initialize the fertility matrix and identity matrix.

        Returns:
        - ft: Fertility matrix.
        - e: Identity matrix with appropriate dimensions.
        """
        ft = np.zeros((self.ages * 2, self.ages * 2))
        ft[0, :self.ages] = np.array(self.f) * self.birth_female

        e = np.zeros((self.ages * 2, self.ages * 2))
        np.fill_diagonal(e[:self.ages, :self.ages], 1)

        return ft, e

    def _calculate_stationary_distribution(self):
        """
        Calculate the stationary distribution vector pi.

        Returns:
        - pi: Stationary distribution vector.
        Calculates and returns the largest eigenvector instead of the first one in the matrix.
        """
        # Compute matrix A
        A = self.Ut[0:self.ages, 0:self.ages] + self.ft[0:self.ages, 0:self.ages]
        # Compute eigenvalues and eigenvectors
        eigenvalues, eigenvectors = np.linalg.eig(A)
        # Calculate the magnitude (L2 norm) of each eigenvector
        magnitudes = np.linalg.norm(eigenvectors, axis=0)
        # Find the index of the largest eigenvector
        largest_eigenvector_index = np.argmax(magnitudes)
        # Get the largest eigenvector
        w = np.real(eigenvectors[:, largest_eigenvector_index])
        # Normalize the largest eigenvector
        w = w / np.sum(w)
        # Compute the stationary distribution pi
        pi = w * A[0, :] / np.sum(w * A[0, :])
        return pi.reshape(-1)

    def _initialize_kin_matrices(self):
        """
        Initialize matrices for different kin types.

        Returns:
        - kin_dict: Dictionary containing initialized kin matrices.
        """
        d = np.zeros((self.ages * 2, self.ages))
        gd = np.zeros((self.ages * 2, self.ages))
        ggd = np.zeros((self.ages * 2, self.ages))
        m = np.zeros((self.ages * 2, self.ages))
        gm = np.zeros((self.ages * 2, self.ages))
        ggm = np.zeros((self.ages * 2, self.ages))
        os = np.zeros((self.ages * 2, self.ages))
        ys = np.zeros((self.ages * 2, self.ages))
        nos = np.zeros((self.ages * 2, self.ages))
        nys = np.zeros((self.ages * 2, self.ages))
        oa = np.zeros((self.ages * 2, self.ages))
        ya = np.zeros((self.ages * 2, self.ages))
        coa = np.zeros((self.ages * 2, self.ages))
        cya = np.zeros((self.ages * 2, self.ages))

        # Initialize the mother matrix with the stationary distribution
        m[:len(self.pi), 0] = self.pi

        return {
            'd': d,
            'gd': gd,
            'ggd': ggd,
            'm': m,
            'gm': gm,
            'ggm': ggm,
            'os': os,
            'ys': ys,
            'nos': nos,
            'nys': nys,
            'oa': oa,
            'ya': ya,
            'coa': coa,
            'cya': cya
        }

    def _propagate_kin(self):
        """
        Propagate kin over time based on the initialized matrices.
        """
        for i in range(0, self.ages - 1):
            self.kin_dict['d'][:, i + 1] = np.dot(self.Ut, self.kin_dict['d'][:, i]) + np.dot(self.ft, self.e[:, i])
            self.kin_dict['gd'][:, i + 1] = np.dot(self.Ut, self.kin_dict['gd'][:, i]) + np.dot(self.ft, self.kin_dict['d'][:, i])
            self.kin_dict['ggd'][:, i + 1] = np.dot(self.Ut, self.kin_dict['ggd'][:, i]) + np.dot(self.ft, self.kin_dict['gd'][:, i])
            self.kin_dict['m'][:, i + 1] = np.dot(self.Ut, self.kin_dict['m'][:, i])
            self.kin_dict['ys'][:, i + 1] = np.dot(self.Ut, self.kin_dict['ys'][:, i]) + np.dot(self.ft, self.kin_dict['m'][:, i])
            self.kin_dict['nys'][:, i + 1] = np.dot(self.Ut, self.kin_dict['nys'][:, i]) + np.dot(self.ft, self.kin_dict['ys'][:, i])

        self.kin_dict['gm'][:self.ages, 0] = np.dot(self.kin_dict['m'][:self.ages, :], self.pi)
        for i in range(0, self.ages - 1):
            self.kin_dict['gm'][:, i + 1] = np.dot(self.Ut, self.kin_dict['gm'][:, i])

        self.kin_dict['ggm'][:self.ages, 0] = np.dot(self.kin_dict['gm'][:self.ages, :], self.pi)
        for i in range(0, self.ages - 1):
            self.kin_dict['ggm'][:, i + 1] = np.dot(self.Ut, self.kin_dict['ggm'][:, i])

        self.kin_dict['os'][:self.ages, 0] = np.dot(self.kin_dict['d'][:self.ages, :], self.pi)
        self.kin_dict['nos'][:self.ages, 0] = np.dot(self.kin_dict['gd'][:self.ages, :], self.pi)
        for i in range(0, self.ages - 1):
            self.kin_dict['os'][:, i + 1] = np.dot(self.Ut, self.kin_dict['os'][:, i])
            self.kin_dict['nos'][:, i + 1] = np.dot(self.Ut, self.kin_dict['nos'][:, i]) + np.dot(self.ft, self.kin_dict['os'][:, i])

        self.kin_dict['oa'][:self.ages, 0] = np.dot(self.kin_dict['os'][:self.ages, :], self.pi)
        self.kin_dict['ya'][:self.ages, 0] = np.dot(self.kin_dict['ys'][:self.ages, :], self.pi)
        self.kin_dict['coa'][:self.ages, 0] = np.dot(self.kin_dict['nos'][:self.ages, :], self.pi)
        self.kin_dict['cya'][:self.ages, 0] = np.dot(self.kin_dict['nys'][:self.ages, :], self.pi)
        for i in range(0, self.ages - 1):
            self.kin_dict['oa'][:, i + 1] = np.dot(self.Ut, self.kin_dict['oa'][:, i])
            self.kin_dict['ya'][:, i + 1] = np.dot(self.Ut, self.kin_dict['ya'][:, i]) + np.dot(self.ft, self.kin_dict['gm'][:, i])
            self.kin_dict['coa'][:, i + 1] = np.dot(self.Ut, self.kin_dict['coa'][:, i]) + np.dot(self.ft, self.kin_dict['oa'][:, i])
            self.kin_dict['cya'][:, i + 1] = np.dot(self.Ut, self.kin_dict['cya'][:, i]) + np.dot(self.ft, self.kin_dict['ya'][:, i])

    def _update_kin_dict(self):
        """
        Update the kin dictionary by removing unwanted kin types and aligning dead kin matrices.
        """
        if self.output_kin is not None:
            for key in list(self.kin_dict.keys()):
                if key not in self.output_kin:
                    del self.kin_dict[key]

        for key, matrix in self.kin_dict.items():
            matrix[self.ages:2*self.ages, :self.ages-1] = matrix[self.ages:2*self.ages, 1:self.ages]
            matrix[self.ages:2*self.ages, self.ages-1] = 0
            self.kin_dict[key] = matrix

    def _create_output_dataframe(self):
        """
        Create an output DataFrame from the kin matrices.

        Returns:
        - output: DataFrame containing kin information.
        """
        output = pd.DataFrame()

        for kin_name, matrix in self.kin_dict.items():
            kin_type = []
            kin_ages = []
            focal_ages = []
            living_kin = []
            dead_kin = []

            for kin_age in range(self.ages):
                for focal_age in range(self.ages):
                    kin_type.append(kin_name)
                    kin_ages.append(kin_age)
                    focal_ages.append(focal_age)
                    living_kin.append(matrix[kin_age, focal_age])
                    dead_kin.append(matrix[kin_age + self.ages, focal_age])

            kin_df = pd.DataFrame({
                "Kin": kin_type,
                'Age of Kin': kin_ages,
                'Age of Focal': focal_ages,
                'Number of Living Kin': living_kin,
                'Number of Dead Kin': dead_kin
            })

            output = pd.concat([output, kin_df], ignore_index=True)

        return output

    def run(self):
        """
        Run the kin propagation model and return the output DataFrame.

        Returns:
        - output: DataFrame containing kin information.
        """
        return self._create_output_dataframe()




In [ ]:
# RUN AND PRINT RESULTS
swe_surv_df = pd.read_csv('swe_surv_2015.csv', names=['age', 'surv_prob'], skiprows=1)
swe_fert_df = pd.read_csv('swe_asfr_2015.csv', names=['age', 'fert_prob'], skiprows=1)

p = swe_surv_df['surv_prob'].to_list()
f = swe_fert_df['fert_prob'].to_list()

kin_model = KinTimeInvariant(p, f)
result = kin_model.run()

result.to_csv('output.csv', index = False)
